In [ ]:
from ctapipe.io import EventSource
from ctapipe import utils
from matplotlib import pyplot as plt
import numpy as np
from ctapipe.utils import get_dataset_path
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from matplotlib import pyplot as plt
from astropy import units as u
%matplotlib inline

In [ ]:
path = utils.get_dataset_path("gamma_prod5.simtel.zst")

In [ ]:
source = EventSource(path, max_events=5)

for event in source:
    print(event.count, event.index.event_id, event.simulation.shower.energy)

In [ ]:
event

In [ ]:
event.r0

In [ ]:
event.r1.tel.keys()

In [ ]:
r0tel = event.r0.tel[3]

In [ ]:
r0tel.waveform.shape

In [ ]:
plt.pcolormesh(r0tel.waveform[1])

In [ ]:
brightest_pixel = np.argmax(r0tel.waveform[0].sum(axis=1))
print(f"pixel {brightest_pixel} has sum {r0tel.waveform[0,1535].sum()}")

In [ ]:
plt.plot(r0tel.waveform[0,brightest_pixel], label="channel 0 (high-gain)")
plt.plot(r0tel.waveform[1,brightest_pixel], label="channel 1 (low-gain)")
plt.legend()

In [ ]:
#from ipywidgets import interact

#@interact
#def view_waveform(chan=0, pix_id=brightest_pixel):
#    plt.plot(r0tel.waveform[chan, pix_id])

In [ ]:
event

In [ ]:
subarray = source.subarray

In [ ]:
subarray

In [ ]:
subarray.peek()

In [ ]:
subarray.to_table()

In [ ]:
subarray.tel[2].optics

In [ ]:
from ctapipe.visualization import CameraDisplay

In [ ]:
tel = subarray.tel[2]

In [ ]:
disp = CameraDisplay(tel.camera.geometry)

In [ ]:
disp = CameraDisplay(tel.camera.geometry)
disp.image = r0tel.waveform[0,:,10]  

In [ ]:
from ctapipe.calib import CameraCalibrator

In [ ]:
calib = CameraCalibrator(subarray=subarray)

In [ ]:
for event in EventSource(path, max_events=5):
    calib(event) # fills in r1, dl0, and dl1
    print(event.dl1.tel.keys())

In [ ]:
event.dl1.tel[3]

In [ ]:
dl1tel = event.dl1.tel[3]

In [ ]:
dl1tel.image.shape

In [ ]:
len(dl1tel.peak_time)

In [ ]:
CameraDisplay(tel.camera.geometry, image=dl1tel.image)

In [ ]:
CameraDisplay(tel.camera.geometry, image=dl1tel.peak_time)

In [ ]:
from ctapipe.image import hillas_parameters, tailcuts_clean

In [ ]:
image = dl1tel.image
mask = tailcuts_clean(tel.camera.geometry, image, picture_thresh=10, boundary_thresh=5)
mask

In [ ]:
CameraDisplay(tel.camera.geometry, image=mask)

In [ ]:
cleaned = image.copy()
cleaned[~mask] = 0

In [ ]:
disp = CameraDisplay(tel.camera.geometry, image=cleaned)
disp.cmap = plt.cm.coolwarm
disp.add_colorbar()
plt.xlim(0.5, 1.0)
plt.ylim(-1.0, 0.0)

In [ ]:
params = hillas_parameters(tel.camera.geometry, cleaned)
print(params)

In [ ]:
disp = CameraDisplay(tel.camera.geometry, image=cleaned)
disp.cmap = plt.cm.coolwarm
disp.add_colorbar()
plt.xlim(0.5, 1.0)
plt.ylim(-1.0, 0.0)
disp.overlay_moments(params, color='white', lw=2)

Going to the processing of RAW DATA

In [ ]:
source = EventSource(get_dataset_path("gamma_prod5.simtel.zst"), max_events=5)

In [ ]:
# so we can advance through events one-by-one
event_iterator = iter(source)

event = next(event_iterator)

In [ ]:
event.r0

In [ ]:
print(event.simulation.shower)

In [ ]:
event = next(event_iterator)

In [ ]:
event.simulation.tel.keys()

In [ ]:
teldata = event.r0.tel[26]
print(teldata)
teldata

In [ ]:
event.simulation.shower.energy

In [ ]:
event.simulation.shower.energy.to('GeV')

In [ ]:
plt.pcolormesh(teldata.waveform[0])  # note the [0] is for channel 0
plt.colorbar()
plt.xlabel("sample number")
plt.ylabel("Pixel_id")

?? In waverform array this is (𝑁𝑐ℎ𝑎𝑛𝑛𝑒𝑙𝑠, 𝑁𝑝𝑖𝑥𝑒𝑙𝑠, 𝑁𝑠𝑎𝑚𝑝𝑙𝑒𝑠) ??


In [ ]:
plt.pcolormesh(teldata.waveform[0])
plt.colorbar()
plt.ylim(700,750)
plt.xlabel("sample number")
plt.ylabel("pixel_id")
print("waveform[0] is an array of shape (N_pix,N_slice) =",teldata.waveform[0].shape)

In [ ]:
trace = teldata.waveform[0][719]
plt.plot(trace, drawstyle='steps')

In [ ]:
for pix_id in range(718,722):
    plt.plot(teldata.waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
plt.legend()

In [ ]:
for pix_id in range(718,723):
    plt.plot(teldata.waveform[0][pix_id],'+-')
plt.fill_betweenx([0,1600],19,24,color='red',alpha=0.3, label='Ped window')
plt.fill_betweenx([0,1600],5,9,color='green',alpha=0.3, label='Signal window')
plt.legend()

In [ ]:
data = teldata.waveform[0]
peds = data[:, 19:24].mean(axis=1)  # mean of samples 20 to 29 for all pixels
sums = data[:, 5:9].sum(axis=1)/(13-8)    # simple sum integration

In [ ]:
phist = plt.hist(peds, bins=50, range=[0,150])
plt.title("Pedestal Distribution of all pixels for a single event")

In [ ]:
plt.plot(sums - peds)
plt.xlabel("pixel id")
plt.ylabel("Pedestal-subtracted Signal")

In [ ]:
# we can also subtract the pedestals from the traces themselves, which would be needed to compare peaks properly
for ii in range(270,280):
    plt.plot(data[ii] - peds[ii], drawstyle='steps', label="pix{}".format(ii))
plt.legend()

In [ ]:
camgeom = source.subarray.tel[24].camera.geometry

In [ ]:
title="CT24, run {} event {} ped-sub".format(event.index.obs_id,event.index.event_id)
disp = CameraDisplay(camgeom,title=title)
disp.image = sums - peds
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)  # autoscale

In [ ]:
for tel in event.r0.tel.keys():
    plt.figure()
    camgeom = source.subarray.tel[tel].camera.geometry
    title="CT{}, run {} event {}".format(tel,event.index.obs_id,event.index.event_id)
    disp = CameraDisplay(camgeom,title=title)
    disp.image = event.r0.tel[tel].waveform[0].sum(axis=1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)

In [ ]:
from scipy import signal
import numpy as np

In [ ]:
pix_ids = np.arange(len(data))
has_signal = sums > 300

widths = np.array([8,]) # peak widths to search for (let's fix it at 8 samples, about the width of the peak)
peaks = [signal.find_peaks_cwt(trace,widths) for trace in data[has_signal] ]

for p,s in zip(pix_ids[has_signal],peaks):
    print("pix{} has peaks at sample {}".format(p,s))
    plt.plot(data[p], drawstyle='steps-mid')
    plt.scatter(np.array(s),data[p,s])

In [ ]:
import ctapipe
from ctapipe.utils.datasets import get_dataset_path
from ctapipe.io import EventSource, EventSeeker
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from matplotlib import pyplot as plt
from astropy import units as u
import numpy as np

%matplotlib inline
plt.style.use("ggplot")

In [ ]:
print(ctapipe.__version__)
print(ctapipe.__file__)

In [ ]:
filename = get_dataset_path("gamma_prod5.simtel.zst")
source = EventSource(filename, max_events=2)

for event in source:
    print(event.index.event_id)

In [ ]:
filename

In [ ]:
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor

image_processor = ImageProcessor(subarray)
muon_processor = MuonProcessor(subarray)

In [ ]:
import astropy.coordinates as c
import astropy.units as u
import matplotlib.pylab as plt
import numpy as np
from ctapipe.coordinates import CameraFrame, EngineeringCameraFrame, TelescopeFrame
from ctapipe.image import hillas_parameters, tailcuts_clean, toymodel
from ctapipe.instrument import SubarrayDescription
from ctapipe.visualization import CameraDisplay

In [ ]:
# load an example camera geometry from a simulation file
subarray = SubarrayDescription.read("dataset://gamma_prod5.simtel.zst")
geom = subarray.tel[100].camera.geometry

# create a fake camera image to display:
model = toymodel.Gaussian(
    x=0.2 * u.m,
    y=0.0 * u.m,
    width=0.05 * u.m,
    length=0.15 * u.m,
    psi="35d",
)

image, sig, bg = model.generate_image(geom, intensity=1500, nsb_level_pe=10)
mask = tailcuts_clean(geom, image, picture_thresh=15, boundary_thresh=5)

geom

In [ ]:
geom

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 4))
CameraDisplay(geom, image=image, ax=ax[0])
CameraDisplay(geom.transform_to(EngineeringCameraFrame()), image=image, ax=ax[1])
CameraDisplay(geom.transform_to(TelescopeFrame()), image=image, ax=ax[2])

In [ ]:
from ctapipe.utils.datasets import get_dataset_path
from ctapipe.io import EventSource
import numpy as np

filename = get_dataset_path("gamma_prod5.simtel.zst")

with EventSource(filename, max_events=1) as source:
    subarray = source.subarray


In [ ]:
subarray.info()

In [ ]:
subarray.to_table()

In [ ]:
dir(subarray)

In [ ]:
subarray.optics_types

In [ ]:
tel = subarray.tel[1]
tel

In [ ]:
dir(tel.camera)

In [ ]:
%matplotlib inline
from ctapipe.visualization import CameraDisplay

CameraDisplay(tel.camera.geometry)

In [ ]:
subarray.tel_index_array